In [1]:
import pickle as pkl
import pandas as pd
import imodels

# this notebooks loads the performances for different models on various datasets into a pandas dataframe

In [2]:
model_comparisons_file = '../imodels/util/evaluate/model_comparisons.pkl'
df = pkl.load(open(model_comparisons_file, 'rb'))['df']

In [3]:
df[df.index.str.contains('Acc')]

,breast-cancer,breast-w,credit-g,haberman,heart,ionosphere,labor,vote
RandomForest (sklearn) Acc.,0.723593,0.964275,0.753,0.702909,0.822222,0.937344,0.893939,0.963218
GradientBoostingClassifier (sklearn) Acc.,0.720206,0.958561,0.760,0.686515,0.788889,0.923099,0.842424,0.954023
MLPClassifier (sklearn) Acc.,0.709800,0.952837,0.661,0.728715,0.822222,0.903421,0.674242,0.956322
SkopeRules Acc.,0.297459,0.655355,0.706,0.738340,0.444444,0.644225,0.718182,0.386207
RuleFit Acc.,0.664368,0.948499,0.744,0.676785,0.774074,0.920322,0.859091,0.951724
FPLasso Acc.,0.692438,0.957122,0.733,0.731676,0.825926,0.886197,0.857576,0.954023
FPSkope Acc.,0.591107,0.806937,0.695,0.734902,0.651852,0.684145,0.824242,0.657471
BRL Acc.,0.702541,0.944265,0.300,0.265098,0.629630,0.866117,0.663636,0.910345
GRL Acc.,0.370357,0.925663,0.700,0.705870,0.377778,0.358592,0.246970,0.043678
OneR Acc.,0.695765,0.905642,0.696,0.705870,0.700000,0.783380,0.784848,0.866667
